In [57]:
n = 3
L = 8*n + 4
t = 1
Ne = Int(L / 2)
function Hamiltonian(L,t)
    H = zeros(2*L, 2*L)
    H[1,L] = -t
    H[L,1] = -t
    H[2L,L+1] = -t
    H[L+1,2L] = -t
    for i=1:2*L
      for j=1:2*L
        if i == j + 1 || i == j - 1
          H[i,j] = -t
        end 
      end
    end
    H[L,L+1]=0
    H[L+1,L]=0
    return H
end

H = Hamiltonian(L,t)

56×56 Matrix{Float64}:
  0.0  -1.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 -1.0   0.0  -1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0  -1.0   0.0  -1.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0  -1.0   0.0  -1.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0  -1.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0  -1.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0  -1.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0  …   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0
  

In [58]:
function Lstate(L)                                 #generate a state of length L
    Lstated, Lstateu, Lstate = zeros(L), zeros(L), zeros(L)
    for j=1:L/4
        Lstateu[rand(1:L)]=1
    end
    for j=1:L/4
        Lstated[rand(1:L)]=-1
    end
    for i=1:L 
        if Lstated[i] == -1 && Lstateu[i] == 1
            Lstate[i] = 2
        else
            Lstate[i] = Lstated[i] + Lstateu[i]
        end
    end
    return Lstate
end

Lstate (generic function with 1 method)

In [59]:
function extendedstate(Lstatevector)               #take a state of length L and make it 2*L
    L = length(Lstatevector)
    State2L = zeros(2*L)
    for j=1:L
        if Lstatevector[j] == -1
            State2L[j + L] = 1
        elseif Lstatevector[j] == 1
            State2L[j] = 1
        elseif Lstatevector[j] == 2
            State2L[j], State2L[j + L] = 1,1
        else
            State2L[j] = 0
        end
    end
    return State2L
end

extendedstate (generic function with 1 method)

In [60]:
function shortstate(ext_state)                    #pass trom 2*L state to L state
    L = Int(length(ext_state)/2)
    sh_state = zeros(L)
    for j=1:L
        if ext_state[j]==1
            sh_state[j]=1
        end
    end
    for j=1:L
        if ext_state[L+j]==1
            if sh_state[j]==1
                sh_state[j]=2
            else
                sh_state[j]=-1
            end
        end
    end
    return sh_state
end


shortstate (generic function with 1 method)

In [67]:
function Pauli(x,i,j)                 #check if state i and j are both occupied
    if x[i] == 1 && x[j] == 1
        return true
    end
    return false
end 

function occupied_states(v)         #generate a vector with positions of occupied states (from 2*L states)
    occ_states = []
    for i = 1:2*L
        if v[i] == 1
            append!(occ_states, i)
        end
    end
    return occ_states
end


function occupied_states_L(v)
    occ_states = []
    L =length(v)
    for i = 1 : L
        if abs(v[i]) == 1
            append!(occ_states, i )
            
        elseif v[i] == 2
            append!(occ_states, i )
        
        end
    end
    return occ_states
end


function move_L(w)
    
    v = copy(w)
    L = length(v)
    i = rand(occupied_states_L(v))
    j = rand(-1:2:1)

    j_a = mod(i+j,L)
    j_a == 0 ? j_a = L : j_a = j_a

        if v[i] != v[j_a] && v[j_a]!=2

            if abs(v[i])==1
            v[j_a] == 0 ? v[j_a] = v[i] : v[j_a] = 2
            v[i] = 0

            elseif v[i]==2    
            spin = rand(-1:2:1)

                if v[j_a] != spin
                v[i] = -spin

                    if abs(v[j_a]) == 1 
                        v[j_a] = 2 
                    elseif v[j_a] == 0 
                        v[j_a] = spin 
                    end

                end    
            end
        end

    return v
    
end


move_L (generic function with 1 method)

In [68]:
function hopping(extended_state)
    hopped_state = copy(extended_state)
    L = length(extended_state)
    x = occupied_states(extended_state)
    hep = rand(x)
    if extended_state(hep + 1) == 1 && extended_state(hep - 1) == 1
        hep = rand(filter!(e -> e != hep,x))
    he = extended_state[hep] 
    if hep != L/2 && hep != 1 && hep != L && hep != L/2+1
            nep = rand([hep + 1,hep - 1])
        elseif hep == 1
            nep = rand([hep + 1, L/2])
        else
            nep = rand([hep - 1, 1])
        end
    else
        if hep != L && hep != L/2 + 1
            nep = rand([hep + 1,hep - 1])
        elseif hep == L/2 + 1
            nep = rand([hep + 1, L])
        else
            nep = rand([hep - 1, L/2 + 1])
        end
    end
        
    hopped_state[hep], hopped_state[nep] = 0, 1
    return hopped_state
end

hopping (generic function with 1 method)

In [69]:
using LinearAlgebra
U = eigvecs(H)  #unitary matrix
Udag = inv(U)   #U^-1
E = eigvals(H)  #eigenvalues of H

56-element Vector{Float64}:
 -2.0000000000000018
 -2.0
 -1.9498558243636503
 -1.949855824363648
 -1.9498558243636477
 -1.949855824363647
 -1.801937735804839
 -1.801937735804838
 -1.8019377358048378
 -1.8019377358048374
 -1.5636629649360605
 -1.56366296493606
 -1.5636629649360592
  ⋮
  1.5636629649360607
  1.5636629649360632
  1.8019377358048378
  1.801937735804838
  1.801937735804838
  1.8019377358048392
  1.9498558243636466
  1.9498558243636466
  1.949855824363648
  1.9498558243636481
  1.999999999999999
  2.0000000000000004

In [70]:
function U1(A, Ne) #take the first Ne columns
    U_1 = A[:,1:Ne]
    return U_1
end

U_1 = U1(U,Ne)

function U_tilde(A,v)                  #takes just the rows related to occupied states, Ne*Ne matrix
    U_tild = []
    U_tild = A[occupied_states(v),:]
    return U_tild
end

U_tilde (generic function with 1 method)

In [73]:
function ratio_det(v, z)
    rat= det(U_tilde(U_1,z))/det(U_tilde(U_1,v))
    return rat
end  



ratio_det (generic function with 1 method)

In [74]:
function n_i(vi)           #counts the number of els in positions vi, returning (# up , # down)
    
    n_up = 0
    n_down = 0
    
        if vi == 1
            n_up += 1
            
        elseif vi == -1
            n_down += 1
                
        elseif vi == 2
            n_down += 1
            n_up += 1
                
        end

return n_up, n_down

end


n_i (generic function with 1 method)

In [75]:
function Jastrow(state,g,v)
            
    D=length(state)        
    
    n1 = 1
    n2 = 0
    
    Spins=0
    Occupancies=0
            
    n1 = n_i(state[D])[1] + n_i(state[D])[2]  
    
    for i = 1 : D
                
        n_up= n_i(state[i])[1]
                
        n_down = n_i(state[i])[2]
                
             
    Spins = Spins + n_up*n_down 
                
        
    n2= n_up + n_down
    n1n2 = n1*n2   
    n1 = n2
                
    Occupancies = Occupancies + n1n2
            
        end
    
    arg=  -g*Spins -v*Occupancies
            
            return exp(arg)
    
    end


Jastrow (generic function with 1 method)

In [76]:
function MC(q0=0,delta=0.1,nsteps=1000,seed=None):
    if seed is not None:
        np.random.seed(seed)
    q=q0
    traj=[]
    for istep in range(nsteps):
        qtry=q+(2*np.random.rand()-1)*delta
        energy_try=0.5*kappa*qtry**2     #1/2 k q^2
        deltae=energy_try-energy
        if deltae<=0.0:          #take acceptance 1 if the energy decrease
            acceptance=1.0
        else:
            acceptance =  ratio**2 * Jastrow_ratio**2   #acceptance 
        if acceptance >=1.0 or acceptance>np.random.rand(): #metropolis rule
            q=qtry
            energy=energy_try
        traj.append(q)
    return np.array(traj)

LoadError: syntax: newline not allowed after ":" used for quoting

In [77]:
#This function calculates the density observable in the odd sites A
function nA(v)
    nA = 0
    for j=1:length(v)
        if j%2 != 0
            nA += sum(n_i(v[j])) 
        end
    end
    return nA*2/length(v)
end

#This function calculates the density observable in the even sites B
function nB(v)
    nB = 0
    for j=1:length(v)
        if j%2 == 0
            nB += sum(n_i(v[j])) 
        end
    end
    return nB*2/length(v)
end

nB (generic function with 1 method)

In [79]:
function Lstate(L)                                 #generate a state of length L
    Lstated, Lstateu, Lstate = zeros(L), zeros(L), zeros(L)
    for j=1:L/4
        Lstateu[rand(1:L)]=1
    end
    for j=1:L/4
        Lstated[rand(1:L)]=-1
    end
    for i=1:L 
        if Lstated[i] == -1 && Lstateu[i] == 1
            Lstate[i] = 2
        else
            Lstate[i] = Lstated[i] + Lstateu[i]
        end
    end
    return Lstate
end

Lstate (generic function with 1 method)

In [102]:
import Random

function MC(q0, g, v, nsteps = 1000)
    Random.seed!(1234)
    state_L = q0
    sumA=0
    sumB=0
    accepted_moves = 0
    for istep = 1:nsteps
        state_2L = extendedstate(state_L)
        state_new_L = move_L(state_L)
        state_new_2L = extendedstate(state_new_L)
        acc =  (ratio_det(state_2L,state_new_2L) * (Jastrow(state_new_L,g,v)/Jastrow(state_L,g,v)))^2       #acceptance 
        if acc >=1.0 || acc>randn()        #metropolis rule
            n_newA= nA(state_new_L)
            #display(n_newA)
            sumA+=n_newA
            n_newB= nB(state_new_L)
            sumB+=n_newB
            state_L = copy(state_new_L)
            accepted_moves += 1
            #compute observables
        end
    end
    return sumA/accepted_moves
end 
state_prova = [1,2,0,1,-1,0,0,1,0,1,-1,2,0,-1,0,-1,0,0,0,0,0,-1,0,0,0,0,0,1]
MC_prova= MC(state_prova,10,2,1000)

MC_res = []
for i = 1:8
    append!(MC_res, MC(state_prova,10,i,1000))
end
MC_res
using Plots
plot(MC_res)

LoadError: ArgumentError: Package Plots not found in current path:
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.


In [23]:
#generation of random state
#This is not taking into account double occupancies
using StatsBase
function randomstate(L)
    state = sample(vcat([1 for j=1:L/4],[-1 for j=1:L/4],[0 for j=1:L/2],[2 for j=1:L/4]), L, replace=false) #sample randomly assigns values of the vector concatenation vcat to a vector of length L without replacement
    return state
end

LoadError: ArgumentError: Package StatsBase not found in current path:
- Run `import Pkg; Pkg.add("StatsBase")` to install the StatsBase package.


In [ ]:
function fromLstateto2L(Lstatevector)
    L = length(Lstatevector)
    State2L = zeros(2*L)
    for j=1:L
        if Lstatevector[j] == -1
            State2L[j + L] = 1
        elseif Lstatevector[j] == 1
            State2L[j] = 1
        elseif Lstatevector[j] == 2
            State2L[j], State2L[j + L] = 1,1
        else
            State2L[j] = 0
        end
    end
    return State2L
end

In [ ]:
a = Lstate(12)

In [ ]:
fromLstateto2L(a)

In [ ]:
if Pauli(v,x[i],x[i]+1) || if (x[i] == Int(len_v/2) && Pauli(x[i],1)) || if (x[i] == len_v && Pauli(x[i],Int(len_v/2+1)))

In [ ]:
function averagedensity(state)
    nprom = oddsum()

In [ ]:
import Pkg; Pkg.add("Plots")